In [36]:
import pandas as pd
import seaborn as sns
from pandas import Timedelta
import warnings
import itertools
from scipy.sparse import csr_matrix, vstack, hstack
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
warnings.simplefilter(action='ignore', category=FutureWarning)
sns.set_theme()
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

def preprocess_dataset(df: pd.DataFrame):
    final_dataset = df.dropna()
    label_encoder = LabelEncoder()
    final_dataset['city'] = label_encoder.fit_transform(final_dataset['city'])
    #final_dataset['user'] = label_encoder.fit_transform(final_dataset['user'])
    
    final_dataset['day']=final_dataset['timestamp'].dt.dayofyear
    final_dataset['hourminute']= final_dataset['timestamp'].dt.hour * 60 + final_dataset['timestamp'].dt.minute
    final_dataset['date_completeness'] = label_encoder.fit_transform(final_dataset['date_completeness'])
    # IMPORTANT DROP FOR LEAKAGE
    final_dataset = final_dataset.drop(columns=['block_duration', 'song_listened'])
    
    final_dataset['session_duration']=final_dataset['session_duration'].apply(lambda x: x.total_seconds())
    final_dataset['user_listen_time']=final_dataset['user_listen_time'].apply(lambda x: x.total_seconds())
    
    final_dataset['premium_user']=final_dataset['premium_user'].astype(int)
    final_dataset['isliked']=final_dataset['isliked'].astype(int)
    final_dataset['isskipped']=final_dataset['isskipped'].astype(int)
    final_dataset['previous_action']=final_dataset['previous_action'].astype(bool).astype(int)
    
    # numeric_cols = final_dataset.select_dtypes(include=['float64']).columns
    # scaler = StandardScaler()
    # final_dataset[numeric_cols] = scaler.fit_transform(final_dataset[numeric_cols])
    
    return final_dataset.drop(columns=['timestamp', 'street', 'name', 'favourite_genres', 'genres', 'session_id', 'track_id', 'user_id', 'id_artist'])

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,ConfusionMatrixDisplay, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import make_pipeline

fdf = preprocess_dataset(pd.read_csv("data/result.csv",index_col=0, parse_dates=['timestamp'], converters={
    'block_duration': lambda x: pd.to_timedelta(x),
    'session_duration': lambda x: pd.to_timedelta(x),
    'user_listen_time': lambda x: pd.to_timedelta(x),
}))
Y = fdf['isskipped']
X = fdf.drop(columns=['isskipped'])
# X = fdf[['user_skip_rate', 'MUSIC1', 'MUSIC2', 'session_duration', 'duration_ms']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=21)

clf = RandomForestClassifier(n_estimators=100)
# clf = SVC(kernel='rbf')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
# Print metrics
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Precision:", precision_score(y_test, y_pred))
# print("Recall:", recall_score(y_test, y_pred))
# print("F1 Score:", f1_score(y_test, y_pred))
# print("ROC-AUC:", roc_auc_score(y_test, y_pred))
# cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
# ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_).plot()

# feature_importances = clf.feature_importances_
# feature_names = X.columns
# importances = pd.Series(feature_importances, index=feature_names)
# importances_sorted = importances.sort_values()
# plt.figure(figsize=(5,2))
# importances_sorted.plot(kind='barh', color='lightblue')
# plt.title('Features Importance in Random Forest Model')
# plt.show()

# Decision Trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,ConfusionMatrixDisplay, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

fdf = preprocess_dataset(pd.read_csv("data/result.csv",index_col=0, parse_dates=['timestamp'], converters={
    'block_duration': lambda x: pd.to_timedelta(x),
    'session_duration': lambda x: pd.to_timedelta(x),
    'user_listen_time': lambda x: pd.to_timedelta(x),
}))
Y = fdf['isskipped']
X = fdf.drop(columns=['isskipped'])
# X = fdf[['user_skip_rate', 'MUSIC1', 'MUSIC2', 'session_duration', 'duration_ms']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=21)

clf = DecisionTreeClassifier()

# clf = SVC(kernel='rbf')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)
import joblib
joblib.dump(clf, 'decision_tree.pkl')